In [74]:
import requests
from bs4 import BeautifulSoup

In [75]:
def find_price(food):
    base_url = 'https://www.gsshop.com/shop/mart/martSearch.gs?'
    params={
        'supCd':'1036018',
        'mq':food,
        'lesq':'417375'
    }

    headers={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
    }

    resp = requests.get(base_url,params=params,headers=headers)
    soup = BeautifulSoup(resp.text)
    li_tags=soup.select('section#searchPrdList li')
    list1 = []
    for li in li_tags[:6]:
        #비고찾기
        img_tag =li.find('span',class_='badge-mart-label')
        if img_tag == None:
            etc=''
        else : 
            etc=img_tag.find('img').get('alt')
        
        prd_name = li.find('dt').text.strip()
        discount = li.find('span',class_='price-discount')
        prd_price = li.find('span',class_='set-price').text
        upper = li.find('del',class_='price-upper')
            
        if discount== None:
            discount = ""
            upper = ""
        else :
            discount = discount.text
            upper = upper.text
            
        list1.append({
            '상품명':prd_name,
            '할인율':discount,
            '원가':upper,
            '판매가':prd_price,
            '비고':etc
        })
    return list1

In [76]:
def get_list(buys):
    bucket=int()
    bucket_list=[]
    #aa=[]
    for buy in buys:
        foodlist = find_price(buy)
        #aa=sorted(flist, key=(lambda x: int(x['판매가'].replace(',','')[:-1])))
        choice={}
        name_sp = buy.split(" ")
        for f in foodlist:
            prd_name = f['상품명']
            buy_cnt=0
            for n in name_sp:
                if n in prd_name :
                    buy_cnt+=1
            if buy_cnt==len(name_sp):
                choice = f
                break;
        if choice == {} :
            choice={
                '상품명':buy,
                '할인율':'',
                '원가':'',
                '판매가':'0원',
                '비고':'제품을 찾을 수 없습니다',
            }
        price= choice.get('판매가')
        bucket+=int(price.replace(',','')[:-1])
        bucket_list.append(choice)
    return (bucket_list,bucket)

In [77]:
def input_prod():
    prod_list=[]
    prod=input('상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):')
    while prod!='' :
        prod_list.append(prod)
        prod=input('상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):')
    return get_list(prod_list)

In [78]:
li1,total=input_prod()

상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):당그
상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):시금치
상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):커피
상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):


In [72]:
import pandas as pd

In [79]:
li_df=pd.DataFrame(li1)
li_df

,상품명,할인율,원가,판매가,비고
0,당그,,,0원,제품을 찾을 수 없습니다
1,[일동후디스]유기농쌀과자 시금치떡뻥 30g,40%,"4,200원","2,520원",행사상품
2,[서울]커피포리묶음 200ml x 3입,10%,"2,660원","2,390원",행사상품
